In [33]:
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())

In [34]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-4o-mini")

In [35]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

history= {}
def get_by_session_id(session_id:str) -> BaseChatMessageHistory:
    if session_id not in history:
        history[session_id] = ChatMessageHistory()
    return history[session_id]
chatbot_with_message_history=RunnableWithMessageHistory(chatbot, get_by_session_id)

In [36]:
session1 = {"configurable": {"session_id": "001"}}
session2 = {"configurable": {"session_id": "002"}}

In [37]:
from langchain_core.messages import HumanMessage
chatbot_with_message_history.invoke([HumanMessage(content="My favorite color is red.")],config=session1)

AIMessage(content="That's great! Red is a vibrant and powerful color often associated with energy, passion, and love. Do you have a specific shade of red that you like, or do you enjoy it in general?", response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 13, 'total_tokens': 53, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-d9749443-d87d-4a83-bfc4-dd28cbd5f9d8-0', usage_metadata={'input_tokens': 13, 'output_tokens': 40, 'total_tokens': 53})

In [38]:
responseFromChatbot = chatbot_with_message_history.invoke(
    "What's my favorite color?",
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'

In [39]:
responseFromChatbot = chatbot_with_message_history.invoke(
    ("HumanMessage","What's my favorite color?"),
    config=session2,
)
responseFromChatbot.content

"I'm not sure what your favorite color is! If you tell me, I'd love to hear about it."

In [40]:
responseFromChatbot = chatbot_with_message_history.invoke(
    ("HumanMessage","What's my favorite color?"),
    config=session1,
)
responseFromChatbot.content

"Your favorite color is red. If you'd like to share more about why you like it or how you use it in your life, I'd love to hear!"

In [41]:
responseFromChatbot = chatbot_with_message_history.invoke(
    ("HumanMessage","My favorite food is Kadi"),
    config=session2,
)
responseFromChatbot.content

"Kadi is delicious! It's a popular dish in Indian cuisine, often made with yogurt and gram flour, and usually served with rice. Do you have a favorite way of preparing it or any particular ingredients you like to add?"

In [42]:
responseFromChatbot = chatbot_with_message_history.invoke(
    ("HumanMessage","Which is my favorite food?"),
    config=session2,
)
responseFromChatbot.content

"You mentioned that your favorite food is Kadi! It's a great choice. Do you enjoy it with any specific sides or accompaniments?"

In [43]:
print(history)

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='My favorite color is red.'), AIMessage(content="That's great! Red is a vibrant and powerful color often associated with energy, passion, and love. Do you have a specific shade of red that you like, or do you enjoy it in general?", response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 13, 'total_tokens': 53, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-d9749443-d87d-4a83-bfc4-dd28cbd5f9d8-0', usage_metadata={'input_tokens': 13, 'output_tokens': 40, 'total_tokens': 53}), HumanMessage(content="What's my favorite color?"), AIMessage(content='Your favorite color is red!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 66, 'total_tokens': 72, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 

In [44]:
session3 = {"configurable": {"abc": "003"}}

CHatbot with Limited Memory

In [60]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
	
def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    if len(messages) < number_of_messages_to_keep:
        return messages  # Return all if less than the limit
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    # x is input & it is a dictionary with key "messages" and value as list of messages along with other information
    # lambda x: limited_messages(x["messages"]) is a function that takes x as input and returns limited_messages(x["messages"])
    # then this messages are passed to prompt
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    # Prompt has messages place holder which is replaced by messages from previous step
    | prompt 
    | chatbot
)
# This is a runnable with message history which creates input where messages are stored in "messages" key
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_by_session_id,
    input_messages_key="messages",
)
session1 = {"configurable": {"session_id": "001"}}
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite vehicles are Vespa scooters.")],
    config=session1,
)
responseFromChatbot.content

"Vespa scooters are indeed a fantastic choice! Their combination of style, practicality, and ease of use makes them a favorite for many. Whether you're navigating through city streets or enjoying a leisurely ride, Vespas offer a unique experience. What do you love most about them? Do you have a particular model in mind?"

In [61]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite city is Pali.")],
    config=session1,
)
responseFromChatbot.content

"Pali is a beautiful city! Known for its rich history and stunning landscapes, it offers a unique blend of cultural heritage and natural beauty. Whether you're interested in exploring historical sites, enjoying local cuisine, or experiencing the vibrant community, there's a lot to appreciate in Pali. What do you love most about the city?"

In [62]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Tell me my favorite color.")],
    config=session1,
)
responseFromChatbot.content

"I don't know your favorite color, but I'd love to hear about it! Do you have a particular color that you love or that resonates with you?"